In [ ]:
# -*- coding: utf-8 -*-
"""Falcon_Interface_Testing.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1Heiim_2G7lSoKJnkfQMOYABRsnUE4rHI

## Finetune Falcon on a Google colab

Let's leverage PEFT library and QLoRA for more memory efficient finetuning.

## Setup

Run the cells below to setup and install the required libraries. For our experiment we will need `accelerate`, `peft`, `transformers`, `datasets` and TRL to leverage the recent [`SFTTrainer`](https://huggingface.co/docs/trl/main/en/sft_trainer). We will use `bitsandbytes` to [quantize the base model into 4bit](https://huggingface.co/blog/4bit-transformers-bitsandbytes). We will also install `einops` as it is a requirement to load Falcon models.

In [ ]:
!nvidia-smi

In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Loading the model

In this section we will load the [Falcon 7B model](https://huggingface.co/tiiuae/falcon-7b), quantize it in 4bit and attach LoRA adapters on it.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, AutoModelForQuestionAnswering


#model_name = "/falcon-7B-instruct-300steps-merged"
model_name = "/falcon-7B-instruct-600steps-merged"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

# Let's also load the tokenizer below

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

# Testing after fine-tuning

In [ ]:
DEVICE = "cuda:0"

# Commented out IPython magic to ensure Python compatibility.
# %%time
prompt = f"""
<human>: How to create a molecule
<assistance>:
""".strip()

# A program that performs a multi-threaded matched pair analysis of a set of structures for
# Last updated on May 15, 2023.

encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)

generation_config = model.generation_config
generation_config.max_new_token = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = generation_config.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config.max_length = 200

with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0]))

Below we will load the configuration file in order to create the LoRA model. According to QLoRA paper, it is important to consider all linear layers in the transformer block for maximum performance. Therefore we will add `dense`, `dense_h_to_4_h` and `dense_4h_to_h` layers in the target modules in addition to the mixed query key value layer.